# Delphi ANOVA

## 파일 불러오기

In [ ]:
# 파일 불러오기
file_path = "/content/drive/MyDrive/KoINISW_Project/설문조사/ANOVA.csv"
df = pd.read_csv(file_path, encoding='cp949')

df.info()

## 데이터 전처리

In [ ]:
# 문항 컬럼명 정리
question_columns = {
    '1-1. 제시된 수업자료 내용(정보)이 정확하고 신뢰할 수 있나요?': 'q1_1',
    '1-2. 수업자료에 내용이 학생들의 수준에 대부분 이해 가능하다고 생각하나요?': 'q1_2',
    '1-3. 수업자료 내용이 역사적 사실 기반으로 개념을 제시하고, 개념에 맞는 설명했나요?': 'q1_3',
    '1-4. 수업자료의 내용이 학생 교육에 활용 가능한 자료인가요?': 'q1_4',
    '1-5. 제시된 스토리텔링 콘텐츠가 수업에 활용 가능한 자료라고 생각하시나요?': 'q1_5',
    '2-1. 삽화 자료가 효과적으로 시각화하였다고 생각하시나요?': 'q2_1',
    '2-2. 삽화 자료가 학습 흥미를 유발하고 있다고 생각하시나요?': 'q2_2',
    '2-3. 삽화 자료가 학생들의 이해력 증진 및 학습 동기 유발에 효과적이라고 생각하나요?': 'q2_3',
    '2-4. 삽화 자료가 수업 내용을 기억하거나 회상하는 데 도움이 된다고 느끼셨나요?': 'q2_4',
    '2-5. 삽화 자료를 포함한 콘텐츠 구성이 교사의 수업 준비를 보다 용이하게 만든다고 생각하시나요?': 'q2_5',
    '3-1. 사이트의 전반적인 화면 구성과 레이아웃이 직관적으로 이해하기 쉬웠나요?': 'q3_1',
    '3-2. 수업자료를 준비할 때 방해가 되지 않고 시각적으로 만족스럽습니까?': 'q3_2',
    '3-3. 콘텐츠 탐색(요약, 삽화생성)이 불편함 없이 자연스럽게 이루어졌나요?': 'q3_3',
    '3-4. 웹사이트 이용 과정에서 별도의 학습 없이도 기능을 쉽게 사용할 수 있다고 생각하시나요?': 'q3_4',
    '3-5. 해당 사이트가 실제로 서비스된다면 사용하실 의향이 있으신가요?': 'q3_5'
}

# 컬럼명 변경
df = df.rename(columns=question_columns)

# 문항 구간 정의
sections = {
    "수업자료 내용 및 적절성 및 충실성": ['q1_1', 'q1_2', 'q1_3', 'q1_4', 'q1_5'],
    "삽화 자료의 효과성": ['q2_1', 'q2_2', 'q2_3', 'q2_4', 'q2_5'],
    "웹 사이트 사용 편의성": ['q3_1', 'q3_2', 'q3_3', 'q3_4', 'q3_5'],
}

## 종사기간-전체 문항 ANOVA분석

In [ ]:
# ANOVA 결과 저장
anova_results = []

# 각 문항 섹션별 반복 분석
for section_name, questions in sections.items():
    for q in questions:
        grouped = df.groupby('종사기간')[q].apply(list)
        f_val, p_val = stats.f_oneway(*grouped)
        anova_results.append({
            "문항 구간": section_name,
            "문항": q,
            "F값": round(f_val, 4),
            "p값": round(p_val, 4),
            "유의미한 차이 여부": "유의함" if p_val < 0.05 else "차이 없음"
        })

# 데이터프레임으로 변환
anova_df = pd.DataFrame(anova_results)
display(anova_df)

## 종사기간별 문항별 평균 ANOVA분석

In [ ]:
# 문항 구간별 평균 컬럼 생성
df['q1_avg'] = df[['q1_1', 'q1_2', 'q1_3', 'q1_4', 'q1_5']].mean(axis=1)
df['q2_avg'] = df[['q2_1', 'q2_2', 'q2_3', 'q2_4', 'q2_5']].mean(axis=1)
df['q3_avg'] = df[['q3_1', 'q3_2', 'q3_3', 'q3_4', 'q3_5']].mean(axis=1)

# 평균 문항에 대해 ANOVA 수행
average_questions = {
    '1번 문항 평균': 'q1_avg',
    '2번 문항 평균': 'q2_avg',
    '3번 문항 평균': 'q3_avg'
}

anova_avg_results = []

for name, col in average_questions.items():
    grouped = df.groupby('종사기간')[col].apply(list)
    f_val, p_val = stats.f_oneway(*grouped)
    anova_avg_results.append({
        "문항": name,
        "F값": round(f_val, 4),
        "p값": round(p_val, 4),
        "유의미한 차이 여부": "유의함" if p_val < 0.05 else "차이 없음"
    })

# 결과 테이블 생성
anova_avg_df = pd.DataFrame(anova_avg_results)
anova_avg_df

,문항,F값,p값,유의미한 차이 여부
0,1번 문항 평균,1.0198,0.4352,차이 없음
1,2번 문항 평균,1.1420,0.3609,차이 없음
2,3번 문항 평균,1.5242,0.1925,차이 없음


## 종사기간별-AI의 인식 ANOVA분석

In [ ]:
# 분석 대상 컬럼 (1번 문항 평균)
target_col = 'q1_avg'

# 종사기간 3그룹 기준 그룹 나누기
group_labels = df['종사기간'].unique()

# 각 그룹에 대해 정규성 + 왜도 출력
for group in group_labels:
    data = df[df['종사기간'] == group][target_col].dropna()

    print(f"그룹: {group}")

    # Shapiro-Wilk 정규성 검정
    shapiro_test = stats.shapiro(data)

    # 왜도 계산
    skewness = stats.skew(data, bias=False)

    print(shapiro_test) # Shapiro-Wilk Test
    print("Skewness (절대값):", abs(skewness))
    print("-" * 40)


📌 그룹: 21~25년차
ShapiroResult(statistic=np.float64(0.7768173600846643), pvalue=np.float64(0.004663673957840439))
Skewness (절대값): 1.732445818495478
----------------------------------------
📌 그룹: 11~15년차
ShapiroResult(statistic=np.float64(0.75), pvalue=np.float64(0.0))
Skewness (절대값): 1.7320508075688943
----------------------------------------
📌 그룹: 16~20년차
ShapiroResult(statistic=np.float64(0.9218988807644082), pvalue=np.float64(0.373064999434204))
Skewness (절대값): 0.5527803454651065
----------------------------------------
📌 그룹: 26~30년차
ShapiroResult(statistic=np.float64(0.7648405196358232), pvalue=np.float64(0.008052752353511694))
Skewness (절대값): 1.1751490085238738
----------------------------------------
📌 그룹: 30년 이상
ShapiroResult(statistic=np.float64(0.629776264554299), pvalue=np.float64(0.0012407259151036264))
Skewness (절대값): 2.0000000000000133
----------------------------------------
📌 그룹: 1~5년차
ShapiroResult(statistic=np.float64(nan), pvalue=np.float64(nan))
Skewness (절대값): nan
----

<ipython-input-87-1733926612>:14: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  shapiro_test = stats.shapiro(data)


## 종사기간cluster-대문항별 ANOVA분석

In [ ]:
custom_group_mapping = {
    '교원미임용자': '미임용~15년차',
    '1~5년차': '미임용~15년차',
    '6~10년차': '미임용~15년차',
    '11~15년차': '미임용~15년차',
    '16~20년차': '16~25년차',
    '21~25년차': '16~25년차',
    '26~30년차': '26년 이상',
    '31년차 이상': '26년 이상'
}
df['종사기간_클러스터'] = df['종사기간'].map(custom_group_mapping)

# 각 클러스터별 인원 수
group_counts_df = df['종사기간_클러스터'].value_counts().sort_index().reset_index()
group_counts_df.columns = ['그룹명', '인원수']

# 평균 문항별 ANOVA 분석
average_questions = {
    '수업자료 내용 및 적절성 평가': 'q1_avg',
    '삽화 자료의 효과성 평가': 'q2_avg',
    '웹사이트 사용 편의성 평가': 'q3_avg'
}

anova_cluster_results = []
for name, col in average_questions.items():
    grouped = df.groupby('종사기간_클러스터')[col].apply(list)
    f_val, p_val = stats.f_oneway(*grouped)
    anova_cluster_results.append({
        "문항(평균점)": name,
        "F값": round(f_val, 3),
        "p값": round(p_val, 3),
        "차이 유무": "유의함" if p_val < 0.05 else "차이 없음"
    })

anova_result_df = pd.DataFrame(anova_cluster_results)

display(group_counts_df)
print()
display(anova_result_df)

,그룹명,인원수
0,16~25년차,21
1,26년 이상,9
2,미임용~15년차,8


,문항(평균점),F값,p값,차이 유무
0,수업자료 내용 및 적절성 평가,0.487,0.619,차이 없음
1,삽화 자료의 효과성 평가,0.265,0.769,차이 없음
2,웹사이트 사용 편의성 평가,0.156,0.856,차이 없음
